In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score 

from tslearn.piecewise import SymbolicAggregateApproximation

from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from pyts.classification import KNeighborsClassifier
import gc

from tslearn.piecewise import PiecewiseAggregateApproximation
from IPython.display import Audio
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

In [16]:
X_train = np.load("DATASETS/TimeSeriesTrimmed/X_train.npy")
X_test = np.load("DATASETS/TimeSeriesTrimmed/X_test.npy")
y_train = pd.read_csv('DATASETS/TimeSeriesTrimmed/y_train.csv')
y_test = pd.read_csv('DATASETS/TimeSeriesTrimmed/y_test.csv')

np.nan_to_num(X_train,copy=False)
np.nan_to_num(X_test,copy=False)
print(X_train.shape, X_test.shape)

(1828, 243200) (624, 243200)


In [19]:
emotion_mapper = {"neutral":0, "calm":1, "happy":2, "sad":3, "angry":4, "fearful":5, "disgust":6, "surprised":7}
y_train= y_train["emotion"].replace(emotion_mapper)
y_test=y_test["emotion"].replace(emotion_mapper)

## Approx

In [17]:
n_paa_segments = 1000
paa = PiecewiseAggregateApproximation(n_segments=n_paa_segments)

X_train_paa = list()
for ts in tqdm(X_train):
    ts = ts[~np.isnan(ts)].reshape(1,-1)
    ts_paa = paa.fit_transform(ts).squeeze()
    X_train_paa.append(ts_paa)
      
X_train=np.array(X_train_paa)
del X_train_paa
gc.collect()

X_test_paa = list()
for ts in tqdm(X_test):
    ts = ts[~np.isnan(ts)].reshape(1,-1)
    ts_paa = paa.fit_transform(ts).squeeze()
    X_test_paa.append(ts_paa)
      
X_test=np.array(X_test_paa)
del X_test_paa
gc.collect()

  0%|          | 0/1828 [00:00<?, ?it/s]

  0%|          | 0/624 [00:00<?, ?it/s]

27

## KNN

In [10]:
clf = KNeighborsClassifier(n_neighbors=12, weights='distance',p=2)
clf.fit(X_train, y_train)

y_pred = list()
for ts in tqdm(X_test):
    ts_pred = clf.predict(ts.reshape(1,-1))
    y_pred.append(ts_pred)


print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred,zero_division=0))

  0%|          | 0/624 [00:00<?, ?it/s]

Accuracy 0.1362179487179487
F1-score [0.         0.09401709 0.         0.26102293 0.         0.
 0.         0.        ]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        48
           1       0.08      0.11      0.09        96
           2       0.00      0.00      0.00        96
           3       0.16      0.77      0.26        96
           4       0.00      0.00      0.00        96
           5       0.00      0.00      0.00        96
           6       0.00      0.00      0.00        48
           7       0.00      0.00      0.00        48

    accuracy                           0.14       624
   macro avg       0.03      0.11      0.04       624
weighted avg       0.04      0.14      0.05       624



## KNN DTW

In [5]:
clf = KNeighborsClassifier(n_neighbors=12,weights="distance", metric='dtw_sakoechiba',n_jobs=-1)
clf.fit(X_train, y_train)


y_pred = list()
for ts in tqdm(X_test):
    ts_pred = clf.predict(ts.reshape(1,-1))
    y_pred.append(ts_pred)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

  0%|          | 0/624 [00:00<?, ?it/s]

Accuracy 0.34615384615384615
F1-score [0.3649635  0.48920863 0.3255814  0.25136612 0.53932584 0.17647059
 0.12195122 0.17073171]
              precision    recall  f1-score   support

           0       0.28      0.52      0.36        48
           1       0.37      0.71      0.49        96
           2       0.37      0.29      0.33        96
           3       0.26      0.24      0.25        96
           4       0.59      0.50      0.54        96
           5       0.30      0.12      0.18        96
           6       0.15      0.10      0.12        48
           7       0.21      0.15      0.17        48

    accuracy                           0.35       624
   macro avg       0.32      0.33      0.30       624
weighted avg       0.34      0.35      0.32       624

